In [1]:
!pip install pytorchvideo torch torchvision opencv-python

In [32]:
import torch
import cv2
import numpy as np
from torch import nn
from pytorchvideo.models.hub import slowfast_r50
from torchvision.transforms import Compose
import torchvision.transforms._transforms_video as transforms

# Load pretrained SlowFast model and strip classifier head
def load_slowfast_model():
    model = slowfast_r50(pretrained=True)
    model.blocks[-1].proj = nn.Identity()  # remove classification head
    model.eval()
    return model

# Video transform: normalize pixel values
def get_transform():
    return Compose([
        transforms.NormalizeVideo(
            mean=[0.45, 0.45, 0.45],
            std=[0.225, 0.225, 0.225]
        )
    ])

# Load video frames and format for SlowFast
def load_video_frames(path, num_frames=32, slowfast_alpha=4):
    cap = cv2.VideoCapture(path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sample_rate = max(total_frames // num_frames, 1)

    frames = []
    for i in range(num_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i * sample_rate)
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (224, 224))
        frames.append(frame)

    cap.release()

    if len(frames) < num_frames:
        raise ValueError(f"Only {len(frames)} frames could be read. Expected {num_frames}.")

    video = np.stack(frames)  # (T, H, W, C)
    video = torch.from_numpy(video).float() / 255.0  # Normalize to [0, 1]
    video = video.permute(3, 0, 1, 2)  # (C, T, H, W)
    video = video.unsqueeze(0)  # (B, C, T, H, W)
    return video

# Prepare video for SlowFast input (two pathways)
def pack_pathway(video, alpha=4):
    fast_pathway = video
    slow_pathway = video[:, :, ::alpha, :, :]
    return [slow_pathway, fast_pathway]

# Main function: extract embedding from video path
def extract_slowfast_embedding(video_path):
    model = load_slowfast_model()
    transform = get_transform()
    video = load_video_frames(video_path)
    video = transform(video.squeeze(0)).unsqueeze(0)
    inputs = pack_pathway(video)

    with torch.no_grad():
        embedding = model(inputs)
    return embedding.squeeze(0)  # shape: (2304,)

# Test path
video_path = "/Users/j/code/bjj_classifier/data/training/pulling_guard/pulling_guard01.mp4"

# Extract and print
embedding = extract_slowfast_embedding(video_path)
print("Embedding shape:", embedding.shape)  # should be torch.Size([2304])
print("Embedding sample:", embedding[:5])    # first 5 values

Embedding shape: torch.Size([2304])
Embedding sample: tensor([0.3356, 0.3462, 0.0682, 0.0396, 0.0000])


In [33]:
name_dict = {"name" : "Jacob"}
name_dict["name"]
!pwd

/Users/j/code/bjj_classifier/data


In [ ]:
import os
data_dict = {}
data_dict["training"] = {}
data_dict["training"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["pulling_guard"][file_name] = embedding

data_dict["training"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/training/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/training/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["training"]["passing_guard"][file_name] = embedding
    
data_dict["test"] = {}
data_dict["test"]["pulling_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["pulling_guard"][file_name] = embedding

data_dict["test"]["passing_guard"] = {}
for file_name in os.listdir("../bjj_classifier/data/test/pulling_guard"):
    if file_name != ".DS_Store":
        video_path = f"/Users/j/code/bjj_classifier/data/test/pulling_guard/{file_name}"
        embedding = extract_slowfast_embedding(video_path)
        data_dict["test"]["passing_guard"][file_name] = embedding        


        
print(data_dict["test"]["passing_guard"])

{'pulling_guard05.mp4': tensor([0.1640, 0.1205, 0.0652,  ..., 0.0228, 0.4845, 0.0738]), 'pulling_guard06.mp4': tensor([0.0325, 0.1148, 0.0159,  ..., 0.0290, 0.1209, 0.0611])}


In [52]:
!pwd

/Users/j/code/bjj_classifier/data


In [80]:
import os

def build_dataset():
    """Build training embeddings dictionary"""
    data_dict = None
    if data_dict is not None:
        return data_dict
            
    data_dict = {"training": {}}
    base_path = "/Users/j/code/bjj_classifier/data/training"
        
    for category in os.listdir(base_path):
        if category.startswith("."):
            continue
        category_path = os.path.join(base_path, category)
        data_dict["training"][category] = {}
            
        if os.path.exists(category_path):
            for file_name in os.listdir(category_path):
                if not file_name.startswith("."):
                    video_path = os.path.join(category_path, file_name)
                    try:
                        embedding = extract_slowfast_embedding(video_path)
                        data_dict["training"][category][file_name] = embedding
                    except Exception as e:
                        print(f"Error processing {video_path}: {e}")

        
    return data_dict

data_dict = build_dataset()

In [35]:
data_dict

{'training': {'double_leg': {'double_leg02.mp4': tensor([0.1253, 0.3977, 0.1187,  ..., 0.1405, 0.2846, 0.0883]),
   'double_leg03.mp4': tensor([0.0390, 0.2602, 0.0733,  ..., 0.0843, 0.7537, 0.0084]),
   'double_leg01.mp4': tensor([0.2441, 0.0821, 0.0000,  ..., 0.0040, 0.1752, 0.1164]),
   'double_leg04.mp4': tensor([0.1485, 0.0234, 0.0248,  ..., 0.5675, 1.0567, 0.0928])},
  'passing_guard': {'passing_guard01.mp4': tensor([0.3466, 0.5936, 0.0662,  ..., 0.0000, 0.0532, 0.0076]),
   'passing_guard02.mp4': tensor([0.2020, 0.8280, 0.2705,  ..., 0.0157, 0.0937, 0.0216]),
   'passing_guard03.mp4': tensor([0.3621, 1.1307, 0.4418,  ..., 1.2560, 0.2893, 0.0245]),
   'passing_guard04.mp4': tensor([0.3466, 0.5936, 0.0662,  ..., 0.0000, 0.0532, 0.0076])},
  'pulling_guard': {'pulling_guard02.mp4': tensor([0.1065, 0.0549, 0.0014,  ..., 0.2788, 0.3420, 0.0307]),
   'pulling_guard03.mp4': tensor([0.0519, 0.2449, 0.0847,  ..., 0.1107, 0.3816, 0.1676]),
   'pulling_guard01.mp4': tensor([0.3356, 0.3462, 

In [36]:
!pip install scikit-learn

In [44]:
!pwd

/Users/j/code/bjj_classifier/data


In [45]:
import torch
import torch.nn.functional as F
import sys

embedding_a = data_dict["training"]["pulling_guard"]["pulling_guard02.mp4"]
embedding_b = data_dict["test"]["pulling_guard"]["pulling_guard05.mp4"]

# Compute cosine similarity between [1, D] vectors
similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_b.unsqueeze(0), dim=1).item()
similarity_percentage = (similarity + 1) / 2 * 100

print(f"Similarity: {similarity_percentage:.2f}%")

KeyError: 'test'

In [ ]:
def perdiction(file_name):
    
    index1 = 0
    
    
    while file_name[index1].isalpha():
        index1 += 1
    

    
    embedding_a = data_dict["test"][file_name[0:len(file_name)-index1+1]][file_name]
    
    embedding_list_pulling = [] #make embedding list for all pulling guard videos
    embedding_list_passing = [] #make embedding list for all passing guard videos

    
    for files in os.listdir("../bjj_classifier/data/training/pulling_guard"): 
        if files != ".DS_Store":
            embedding_list_pulling.append(data_dict["training"]["pulling_guard"][files])

    for files in os.listdir("../bjj_classifier/data/training/passing_guard"): 
        if files != ".DS_Store":
            embedding_list_passing.append(data_dict["training"]["passing_guard"][files])

    
    sum1 = 0
    index2 = 0
    for embeddings in embedding_list_pulling:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_pulling[index2].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum1 += similarity_percentage
        index2 += 1

    average_pulling = sum1/len(embedding_list_pulling)
    print(average_pulling)

    sum2 = 0
    index3 = 0
    for embeddings in embedding_list_passing:
        similarity = F.cosine_similarity(embedding_a.unsqueeze(0), embedding_list_passing[index3].unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) / 2 * 100
        sum2 += similarity_percentage
        index3 += 1
        
    average_passing = sum2/len(embedding_list_passing)
    print(average_passing)
    
    if average_passing < average_pulling:
        return "This is pulling guard"
    elif average_passing > average_pulling:
        return "This is passing guard"




In [171]:
#IMPORTANT: .keys() function may be really important
'''
for dir_name in data_dict["training"].keys():
    print(dir_name)
    for files in data_dict["training"][dir_name].keys():
        for embeddings in data_dict["training"][dir_name][files]:
            if embeddings and not dir_name.startswith("."):
                print(embeddings)
'''
data_dict["training"].keys()
#data_dict["training"]["pulling_guard"].keys()
#data_dict["training"]["pulling_guard"]["pulling_guard04.mp4"]

dict_keys(['double_leg', 'passing_guard', 'pulling_guard'])

In [194]:
import string
def prediction06(file_name):
    # Step 1: Extract test embedding
    category = extract_directory_name(file_name)
    input_embedding = extract_slowfast_embedding(f"../data/test/{category}/{file_name}")
    base_path = "../data/training"
    
    
    dir_dict = {
        dir_name: make_embedding_list(dir_name, base_path)
        for dir_name in os.listdir(base_path)
        if not dir_name.startswith(".")
    }
        
    # Compute similarity scores
    embedding_sim_dict = {
        dir_name: max_similarity(input_embedding, embeddings)
        for dir_name, embeddings in dir_dict.items()
        if embeddings
    }
    
    # Step 4: Return best match description
    return compare(embedding_sim_dict)



def extract_directory_name(file_name):
    base_name = os.path.splitext(file_name)[0]  # removes file extension
    index = 0
    while index < len(base_name) and base_name[index].isalpha() or base_name[index] == '_':
        index += 1

    return base_name[:index]

def make_embedding_list(dir_name, base_path="../data/training"):
    embedding_list = []
    dir_path = os.path.join(base_path, dir_name)
    for file_name in os.listdir(dir_path):
        if file_name.startswith("."):
            continue  # skip hidden files like .DS_Store
        embedding = data_dict["training"].get(dir_name, {}).get(file_name)
        if embedding is not None:
            embedding_list.append(embedding)

    return embedding_list

def max_similarity(input_embedding, embedding_list):
    max_percentage = 0.0

    for embedding in embedding_list:
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), embedding.unsqueeze(0), dim=1).item()
        similarity_percentage = (similarity + 1) * 50  # cleaner math
        max_percentage = max(max_percentage, similarity_percentage)

    print(max_percentage)
    return max_percentage

def compare(embedding_sim_dict):
    if not embedding_sim_dict:
        return "No data provided."

    # Find the key with the highest similarity
    best_match = max(embedding_sim_dict, key=embedding_sim_dict.get)

    # Replace punctuation with spaces and split into words
    clean_name = ''.join(c if c not in string.punctuation else ' ' for c in best_match).split()

    # Join only the first two words (if they exist)
    final_string = ' '.join(clean_name[:2])

    return f"This is {final_string}"






In [195]:
print(prediction06("passing_guard05.mp4"))

80.55141568183899
83.88126194477081
80.07263243198395
This is passing guard


In [430]:
import math
import numpy as np

def find_num(num, array, type_change):
    if type_change == 0:
        middle = math.floor((len(array)-1)/2)
    elif type_change == 1:
        middle = math.ceil((len(array)-1)/2)

    
    if array[middle] == num:
        return num
    
    
    if array[middle] > num:
        new_num = find_num(num, array[:middle], 0)
        return new_num
    elif array[middle] < num:
        new_num = find_num(num, a[middle+1:], 1)
        return new_num

In [436]:
a = [1,2,3,4,5,6,7]
b = [1,2,3,4,5,6]
print(find_num(7, a, 0))


RecursionError: maximum recursion depth exceeded in comparison

In [441]:
c = [5,6,7]
middle = 1
c[middle+1:]

[7]